In [1]:
import json
import pandas as pd
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
with open("obligation_data/PBG & Retention money annotations.json", 'r', encoding="utf-8") as f:
    lines = json.load(f)

In [3]:
dataset = []
for line in lines:
    context = line['data']['clause_text']
    annotation = line['annotations']
    payment_terms = []
    all_labels=[]
    for annot in annotation:
        result = annot['result']
        for res in result:
            labels = res['value']['labels']      
            if labels[0] == "What is the amount for retention money?":
#                 print(res)
                value = res['value']
                start_id = value['start']
                end_id = value['end']
                payment_terms.append([start_id, end_id])
                all_labels.append(labels[0])
#             elif labels[0] == "What is the EMD amount?" and j<100:
#                 value = res['value']
#                 start_id = value['start']
#                 end_id = value['end']
#                 payment_terms.append([start_id, end_id])
#                 all_labels.append(labels[0])
#                 j+=1 
        if len(payment_terms) > 0:
            dataset.append([payment_terms, context, all_labels])
print(len(dataset))

33


In [4]:
# splitting the json into words and labels. 
complete_data = []
#set of all words in our dataset
vocab = set()

all_context = []
#docanno answers
doccano_answers = []
low = []
words_context = []
x = []
y = []

#iterate over every data point (sentence - corpus of labelled text) to extract words
for data in dataset: 
    #list to hold all the labels
    labels = []
    #list to hold the extracted answer's class
    extracted_class = []
    #extract the text and annotations from the each data point
    text = data[1]
    indices = data[0]
    label = data[2]
    indices = sorted(indices)
    #print(ord_dict)
    words_context = text.split()             
    #print(words_context) 
    words_inside = []
    words_outside_3 = []
    all_words = []
    words_middle_entities = []
    label_check = []
    
    first = indices[0][0]
    last = indices[len(indices)-1][1]
    outside_entity1 = text[:first]
    words_oe1 = outside_entity1.split()

    outside_entity2 = text[last+1:]
    words_oe2 = outside_entity2.split()

    for i in range(len(indices)):
        start = indices[i][0]
        end = indices[i][1]
        inside_entity = text[start:end+1]
        words_ie = inside_entity.split()
        words_inside.append(words_ie)
#     print(words_inside)
#     print(len(words_inside), words_inside)
#     print(words_inside)
    #the outside entity in between the context.
    for j in range(len(indices)-1):
        end = indices[j+1][0] 
        start = indices[j][1]
        #print(start,end)
        outside_entity3 = text[start+1:end-1]
        words_oe3 = outside_entity3.split()
        words_outside_3.append(words_oe3)
#     print(len(words_outside_3), words_outside_3)
#     print(words_outside_3)
    label_oe1 = []
    label_oe2 = []
    label_oe3s = []
    label_ies = []
    label_middle_entities = []
    #print(words_inside)
    
    for i in range(len(words_oe1)):
        label_oe1.append('O')
    for i in range(len(words_oe2)):
        label_oe2.append('O')
        
    k = 0  
    for i in range(len(words_inside)):
        label_ie = []
        flag = 0
        for j in range(len(words_inside[i])):
            if label[i] == "What is the amount for retention money?":
                label_ie.append('I')
            else:
                label_ie.append('O')
        label_ies.append(label_ie)
        k+=1
    print(label_ies)
        
    for words_oe3 in words_outside_3:
        label_oe3 = []
        for i in range(len(words_oe3)):
            label_oe3.append('O')
        label_oe3s.append(label_oe3)
#     print(len(label_oe3s), len(label_ies))
    
    for i in range(len(label_oe3s)):
        label_middle_entities += label_ies[i]
        label_middle_entities += label_oe3s[i]
    label_middle_entities += label_ies[-1]
    
    for i in range(len(words_outside_3)):
        words_middle_entities += words_inside[i]
        words_middle_entities += words_outside_3[i]
    words_middle_entities += words_inside[-1]
    
    labels =  label_oe1 + label_middle_entities + label_oe2
    all_words = words_oe1 + words_middle_entities + words_oe2 
    #print(len(labels), len(all_words))
#     print(all_words)
    words_ies = " "
    for i in range(len(words_inside)):
        if 'I' in label_ies[i]:
            words_ies += " " + str(" ".join(words_inside[i]))

    if(len(labels)!=0):
        all_context.append(text)
        complete_data.append((words_context,labels))
        x.append(all_words)
        y.append(labels)
        doccano_answers.append(words_ies)
    else:
        x.append(all_words)
        y.append(['O']*len(words_context))
        doccano_answers.append([])

print(y)

[['I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I']]
[['I', 'I', 'I', 'I', 'I', 'I', 

In [5]:
contexts = [data[1] for data in dataset]

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings
model_name = "sentence-transformers/all-mpnet-base-v2"
hf = HuggingFaceEmbeddings(model_name=model_name)

In [7]:
import pinecone
from langchain.vectorstores import Pinecone, Milvus

pinecone.init(api_key="653a0a01-08d9-4684-9a85-1f10ed6ac9e8", environment="asia-southeast1-gcp")
# Define the index settings
index_name = "contract-search"
index_dims = 768  # Dimensionality of the embeddings
index_type = "cosine"  # Similarity metric for the index

In [8]:
def chunk_tokenize(text, chunk_size):
    chunks = []
    num_chunks = len(text) // chunk_size
    for i in range(num_chunks):
        start = i * chunk_size
        end = start + chunk_size
        chunk_span = text[start:end]
        chunks.append(chunk_span)
        
    if len(text) % chunk_size != 0:
        chunk = text[num_chunks * chunk_size:]
        chunks.append(chunk)

    return chunks

In [9]:
query = "What is the amount for retention money?"

In [17]:
actual_answers = []
i = 0
for context in contexts[:2]:
    i +=1
    chunks = chunk_tokenize(context, 1000)
    docsearch = Pinecone.from_texts(chunks, hf, index_name=index_name)
    query_doc = docsearch.similarity_search(query, k=3)
    print(query_doc)
    doc = [d.page_content for d in query_doc]
    best_answer = extract_answers(query, query_doc)
    ans = best_answer["answer"]
#     for d in doc:
#         sentences = sent_tokenize(d)
#         answer = []
#         for sent in sentences:
#             if ans in sent:
#                 answer.append(sent)
#             else:
#                 continue  
#         actual_answers.append(answer)
#     print(i)
    

[Document(page_content='Retention Money\n(i) From every payment for Works due to the Contractor in accordance with the\nprovisions of Clause 19.5, the Authority shall deduct 6% (six per cent) thereof as\nguarantee money for performance of the obligations of the Contractor during the\nConstruction Period (the “Retention Money”)', metadata={}), Document(page_content=' 15 (fifteen) days of the date of issue of the Completion Certificate, the Authority shall refund the balance of Retention Money remaining with the Authority after adjusting the amounts appropriated under the provisions of Clause 7.5 (ii).\n(iv) The Parties agree that in the event of Termination of this Agreement, the Retention Money specified in this Clause 7.5 shall be treated as if they are Performance Security and shall be reckoned as such for the purposes of Termination Payment under Clause 23.6.', metadata={}), Document(page_content='5 (fifteen) days of the date of issue of the Completion Certificate, the\nAuthority sh

In [13]:
print(doccano_answers[1])

  Retention money shall be deducted by the Engineer-in-Charge from the interim bills of the Contractor @ 3% (three percent) of the total value of each bill of the work done (including those of price variation) towards Security Deposit.


In [14]:
from transformers import pipeline
model_name = "deepset/electra-base-squad2"
qa_model = pipeline(tokenizer=model_name, model=model_name, task = "question-answering")

In [15]:
def extract_answers(query, query_doc):
    query_doc = [d.page_content for d in query_doc]
    results = []
    for doc in query_doc:
        answer = qa_model(question=query, context=doc)
        #adding the context info to the answer dict
        answer['context'] = doc
        results.append(answer)
    sorted_results = max(results, key=lambda x: x["score"])
    return sorted_results

In [37]:
print(actual_answer)

['Retention Money\n(i) From every payment for Works due to the Contractor in accordance with the\nprovisions of Clause 19.5, the Authority shall deduct 6% (six per cent) thereof as\nguarantee money for performance of the obligations of the Contractor during the\nConstruction Period (the “Retention Money”)']
